# Word2Vec: A Complete Deep Dive from Scratch

Let me build this up systematically, explaining every concept, algorithm, and design decision.

---

## Part 1: The Motivation - Why Word2Vec?

### The Limitations of Previous Approaches

**Recall LSA/SVD:**
- Operates on entire corpus at once (global statistics)
- Expensive to update (must recompute SVD)
- Requires huge matrix factorization
- Linear algebra operations scale poorly

**One-hot encoding:**
```
cat  = [1, 0, 0, 0, 0, ...]
dog  = [0, 1, 0, 0, 0, ...]
meow = [0, 0, 1, 0, 0, ...]
```

Problems:
- Every word is equally distant from every other word
- No semantic information whatsoever
- Vocabulary-sized vectors (50,000+ dimensions)

### The Word2Vec Revolution (2013)

**Key insight by Tomas Mikolov et al.**: What if we predict words from their context (or context from words) using a neural network? The network's internal representations would naturally capture semantic meaning!

**Core philosophy**: "You shall know a word by the company it keeps" (Firth, 1957)

---

## Part 2: The Fundamental Idea

### Dense Word Embeddings

Word2Vec learns to represent each word as a **dense vector** (typically 100-300 dimensions):

```
cat  = [0.2, -0.4, 0.7, 0.1, ..., 0.3]  # 300 numbers
dog  = [0.3, -0.3, 0.6, 0.2, ..., 0.4]  # 300 numbers
meow = [0.1, -0.5, 0.8, 0.0, ..., 0.2]  # 300 numbers
```

**Why this is better:**
1. **Dense**: Every dimension has a value (no sparsity)
2. **Low-dimensional**: 300 << 50,000
3. **Semantic**: Similar words have similar vectors
4. **Algebraic**: Vector arithmetic captures relationships

### The Distributional Hypothesis in Action

Words appearing in similar contexts should have similar embeddings:

```
"The ___ is sleeping on the mat"
```

Could be: cat, dog, hamster, rabbit → all get similar embeddings

```
"The ___ flew over the building"
```

Could be: bird, plane, helicopter → different cluster of similar embeddings

---

## Part 3: Two Architectures

Word2Vec has two main architectures. Let me explain both thoroughly.

## Architecture 1: CBOW (Continuous Bag of Words)

### The Core Task

**Given context words, predict the center word**

Example sentence: "The quick brown fox jumps"
- Context: ["The", "quick", "brown", "jumps"]
- Target: "fox"

### Why "Bag of Words"?

The order of context words doesn't matter - we just take their average. "Quick brown" = "brown quick" for CBOW.

**Justification**: Simplifies the model and makes it faster. Word order information is sacrificed for computational efficiency.

### The Architecture (Detailed)

Let me walk through the exact neural network structure:

#### **Components:**

1. **Input layer**: One-hot encoded context words
   - Vocabulary size V (e.g., 50,000)
   
2. **Hidden layer** (the embedding layer): 
   - Size N (e.g., 300 dimensions)
   - No activation function! (linear)
   - This is where embeddings live
   
3. **Output layer**: Probability distribution over vocabulary
   - Size V
   - Softmax activation

#### **Weight Matrices:**

- **W_input** (V × N): Input-to-hidden weights
  - Each row is the embedding vector for a word
  - This is the **word embedding matrix** we want to learn!
  
- **W_output** (N × V): Hidden-to-output weights
  - Each column represents a word in the output space
  - Often called the "context matrix"

### Forward Pass (Step by Step)

**Given**: Context words w₁, w₂, ..., wc (c = context size)

**Step 1**: Convert each context word to one-hot vector
```
w₁ → x₁ = [0, 0, ..., 1, ..., 0]  (1 at position of w₁)
w₂ → x₂ = [0, 0, ..., 1, ..., 0]  (1 at position of w₂)
```

**Step 2**: Look up embedding for each context word
```
v₁ = W_input[w₁]  (just extract row w₁ from W_input)
v₂ = W_input[w₂]
...
vc = W_input[wc]
```

**Step 3**: Average the context embeddings
```
h = (v₁ + v₂ + ... + vc) / c
```

**Justification for averaging**: Equal weight to all context words. Simple and works well. Creates a "center of mass" in embedding space.

**Step 4**: Compute output scores
```
u = W_output^T · h
```
Result: u is a V-dimensional vector (one score per vocabulary word)

**Step 5**: Apply softmax to get probabilities
```
p(word_i | context) = exp(u_i) / Σⱼ exp(u_j)
```

**Justification for softmax**: Converts scores to valid probability distribution (sums to 1, all positive).

### The Learning Objective

**Goal**: Maximize the probability of predicting the correct center word

**Loss function** (negative log-likelihood):
```
L = -log p(w_center | context)
  = -u_center + log(Σⱼ exp(u_j))
```

We want to:
- Increase score for correct word (u_center)
- Decrease scores for incorrect words (denominator)

### Backpropagation

**The gradient flow**:

1. Output layer gradient:
```
∂L/∂u_i = p(word_i | context) - 1[i = center]
```
Where 1[...] is 1 if true, 0 otherwise.

**Interpretation**: Error = predicted probability - true probability (1 for correct word, 0 for others)

2. Update W_output:
```
∂L/∂W_output = h · (p - t)^T
```
Where p = predicted probabilities, t = target (one-hot)

3. Hidden layer gradient:
```
∂L/∂h = W_output · (p - t)
```

4. Update W_input (the embeddings!):
```
∂L/∂W_input[wᵢ] = (∂L/∂h) / c  for each context word wᵢ
```

**Key insight**: Only the embeddings of words that appeared in the context get updated! This is efficient.

---

## Architecture 2: Skip-gram

### The Core Task

**Given center word, predict context words**

This is the **reverse** of CBOW!

Example sentence: "The quick brown fox jumps"
- Input: "fox"
- Targets: ["The", "quick", "brown", "jumps"]

### Why Skip-gram?

**Justification**: Skip-gram works better for:
- Rare words (gets more training examples per occurrence)
- Smaller datasets
- Learning more precise embeddings

CBOW works better for:
- Frequent words
- Larger datasets
- Faster training

### The Architecture

**Components:**

1. **Input layer**: One-hot encoding of center word (V dimensions)
2. **Hidden layer**: Embedding layer (N dimensions, no activation)
3. **Output layer**: C separate softmax layers (one per context position)

Or more commonly: **Single output layer** that predicts one context word at a time.

### Forward Pass

**Given**: Center word w_center

**Step 1**: Get embedding
```
h = W_input[w_center]
```

**Step 2**: Compute output scores for each context position
```
For each position i in context:
    u^(i) = W_output^T · h
    p(w | w_center) = softmax(u^(i))
```

### The Learning Objective

**Goal**: Maximize probability of predicting all context words

**Loss function**:
```
L = -Σᵢ log p(w_context_i | w_center)
```

We process multiple context words, so we sum their losses.

### Skip-gram with Multiple Contexts

For sentence: "The quick brown fox jumps over"
Center word: "fox"
Window size: 2

Training pairs generated:
- (fox, The)
- (fox, quick)
- (fox, brown)
- (fox, jumps)

**Each pair is a separate training example!**

**Why this helps rare words**: If "fox" appears once, we get 4 training examples from that single occurrence. CBOW would give us only 1.

---

## Part 4: The Computational Problem

### The Softmax Bottleneck

The softmax denominator is the killer:

```
p(w | context) = exp(u_w) / Σⱼ₌₁^V exp(u_j)
```

We must sum over the **entire vocabulary** (V = 50,000+) for every prediction!

**Computational cost**: O(V) per training example
**With millions of training examples**: Completely impractical!

This is why we need approximations.

---

## Part 5: Solution 1 - Hierarchical Softmax

### The Idea

Instead of computing probability over V words directly, organize vocabulary as a **binary tree**.

**Key insight**: Any word can be reached by a path of log₂(V) binary decisions.

### The Tree Structure

```
                    root
                   /    \
              left/      \right
                /          \
           node1           node2
           /   \           /   \
        word1 word2    word3  word4
```

**Huffman tree**: More frequent words get shorter paths (fewer decisions).

**Justification**: Frequent words are predicted more often, so making them faster improves overall speed.

### How It Works

Instead of V-way classification, we make log₂(V) binary classifications.

**Each node in the tree has a vector** (same dimension as embeddings).

**Probability of a word**:
```
p(w | context) = Πᵢ₌₁^L p(decision_i | context)
```

Where L = length of path to word w

**Binary decision at node n**:
```
p(left | h, n) = σ(v_n^T · h)
p(right | h, n) = 1 - σ(v_n^T · h)
```

Where σ = sigmoid function, v_n = vector for node n

### Computational Savings

- **Before**: O(V) operations per prediction
- **After**: O(log V) operations per prediction

For V = 50,000: 50,000 → 16 operations!

**Trade-off**: Slightly more complex implementation, but massive speedup.

---

## Part 6: Solution 2 - Negative Sampling (Most Popular!)

### The Brilliant Insight

**Question**: Do we really need to distinguish the target word from ALL other words?

**Answer**: No! Just distinguish it from a **small sample** of "negative" words.

### The Objective

Original softmax objective:
```
maximize: log p(w_center | context)
```

Negative sampling objective:
```
maximize: log σ(u_center) + Σₖ₌₁^K log σ(-u_k)
```

Where:
- u_center = score for correct word
- u_k = score for k randomly sampled "negative" words
- σ = sigmoid function
- K = number of negative samples (typically 5-20)

### What Does This Mean?

We're training the model to:
1. **Give high score** to the actual context word (log σ(u_center))
2. **Give low score** to random words that didn't appear (log σ(-u_k))

**Binary classification**: "Is this word in the context?" Yes for 1 word, No for K words.

### Why This Works

**Statistical justification**: If we sample negatives according to word frequency, this approximates the full softmax in expectation.

**Intuitive justification**: The model learns to distinguish real word pairs from noise. If it can do this well, it has learned meaningful embeddings!

### Negative Sampling Distribution

**How to sample negative words?**

Original paper uses: P(w) ∝ freq(w)^(3/4)

**Why the 3/4 power?**

1. **freq(w)^1**: Proportional to frequency
   - Problem: Very common words (like "the") sampled too often
   
2. **freq(w)^0**: Uniform sampling
   - Problem: Rare words sampled as often as common words (unnatural)
   
3. **freq(w)^(3/4)**: Sweet spot!
   - Rare words sampled more than their frequency
   - Common words sampled less than their frequency
   - Balances the distribution

**Mathematical effect**: 
- Word with frequency 100 → relative probability ≈ 31.6
- Word with frequency 10 → relative probability ≈ 5.6
- Ratio: 31.6/5.6 ≈ 5.6 (instead of 10)

More balanced!

### Computational Cost

- **Full softmax**: O(V) operations
- **Negative sampling**: O(K) operations

For K = 10 and V = 50,000: **5,000× speedup!**

---

## Part 7: Training Details

### Context Window

**Window size**: How many words on each side to consider as context?

Example with window = 2:
```
"The quick brown fox jumps over the lazy dog"
                 ↑
            Center: fox
Context: [quick, brown, jumps, over]
```

**Common values**: 5-10 words

**Dynamic windows**: Randomly sample window size from 1 to max for each center word.

**Justification**: 
- Nearby words more important (syntax, strong relationships)
- Distant words provide semantic information
- Dynamic windows give more weight to closer words naturally

### Subsampling Frequent Words

Very common words (the, a, is) appear too often and provide less information.

**Subsampling formula**: Discard word w with probability:

```
P(discard w) = 1 - √(t / freq(w))
```

Where t is a threshold (typically 10^-5), freq(w) is word frequency.

**Example**:
- "the" (freq = 0.1): P(discard) ≈ 0.97 (discard 97% of time!)
- "fox" (freq = 0.0001): P(discard) ≈ 0 (keep it)

**Justification**:
- Speeds up training significantly
- Improves embeddings (less dominated by common words)
- Rare words get relatively more attention

### Learning Rate

Start high (0.025), decay linearly to minimum (0.0001) over training.

**Justification**: Early - make big moves to find good regions. Late - fine-tune with small adjustments.

### Epochs

Typically 5-15 passes through the corpus.

---

## Part 8: The Magic - Semantic Relationships

### Vector Arithmetic

The famous property of Word2Vec:

```
king - man + woman ≈ queen
paris - france + germany ≈ berlin
walking - walk + swim ≈ swimming
```

### Why Does This Work?

**Geometric interpretation**: The model learns that certain **directions** in embedding space represent relationships:

- Gender direction: king → queen (same as man → woman)
- Country-capital direction: france → paris (same as germany → berlin)
- Verb tense direction: walk → walking (same as swim → swimming)

**Mathematical explanation**:

Words that appear in similar contexts get similar embeddings. But more specifically:

- "king" appears with: [royal, kingdom, throne, male, ...]
- "queen" appears with: [royal, kingdom, throne, female, ...]
- "man" appears with: [male, person, ...]
- "woman" appears with: [female, person, ...]

The difference (king - man) captures "royalty" after removing "male-person."
Adding "woman" brings back "female-person."
Result: queen!

### Cosine Similarity

To find similar words:

```
similarity(v₁, v₂) = (v₁ · v₂) / (||v₁|| ||v₂||)
```

**Why cosine?**
- Ignores magnitude (long vs short vectors)
- Focuses on direction (meaning)
- Range: [-1, 1] where 1 = identical direction

---

## Part 9: Implementation Walkthrough

Let me show you a simplified implementation to make everything concrete.

### Pseudocode for Skip-gram with Negative Sampling

```python
# Initialize
W_input = random_matrix(V, N)  # V = vocab size, N = embedding dim
W_output = random_matrix(V, N)

for epoch in epochs:
    for sentence in corpus:
        for position, center_word in enumerate(sentence):
            
            # Get context words
            window = random_int(1, max_window)
            context_words = sentence[position-window : position+window]
            context_words.remove(center_word)
            
            # Subsampling (skip frequent words sometimes)
            if random() < discard_probability(center_word):
                continue
            
            # Get center word embedding
            v_center = W_input[center_word]  # Shape: (N,)
            
            for context_word in context_words:
                
                # Positive example
                v_context = W_output[context_word]
                score_pos = dot(v_center, v_context)
                loss = -log(sigmoid(score_pos))
                
                # Gradient for positive
                grad = (sigmoid(score_pos) - 1)
                W_input[center_word] -= lr * grad * v_context
                W_output[context_word] -= lr * grad * v_center
                
                # Negative examples
                neg_words = sample_negatives(K, vocabulary)
                for neg_word in neg_words:
                    v_neg = W_output[neg_word]
                    score_neg = dot(v_center, v_neg)
                    loss += -log(sigmoid(-score_neg))
                    
                    # Gradient for negative
                    grad = sigmoid(score_neg)
                    W_input[center_word] -= lr * grad * v_neg
                    W_output[neg_word] -= lr * grad * v_center
```

### Key Implementation Details

1. **Which matrix to use as final embeddings?**
   - W_input: Most common choice
   - Average of W_input and W_output: Sometimes better
   - Concatenate both: Doubles dimensions but captures more

2. **Vocabulary preprocessing**:
   - Remove very rare words (< 5 occurrences)
   - Replace rare words with <UNK> token
   - Add special tokens: <PAD>, <START>, <END>

3. **Initialization**:
   - Random uniform: [-0.5/N, 0.5/N]
   - Small random values prevent saturation

---

## Part 10: Advantages and Limitations

### Advantages of Word2Vec

1. **Speed**: Much faster than LSA (local context, not global matrix)
2. **Incremental**: Can update with new text without retraining from scratch
3. **Semantic quality**: Captures analogies and relationships
4. **Dense representations**: No sparsity, efficient storage
5. **Transfer learning**: Pre-trained embeddings work well for many tasks

### Limitations

1. **Fixed vocabulary**: New words (out-of-vocabulary) have no embeddings
   - **Solution**: Use subword embeddings (FastText)

2. **Single embedding per word**: Polysemy problem
   - "bank" (financial) vs "bank" (river) → same embedding!
   - **Solution**: Contextual embeddings (ELMo, BERT)

3. **No word order**: Bag of context words
   - "not good" vs "good" might be too similar
   - **Solution**: Use models that respect order (LSTM, Transformer)

4. **Short context**: Only nearby words matter
   - Long-range dependencies missed
   - **Solution**: Larger windows, or document-level models

5. **Training instability**: Sensitive to hyperparameters
   - Requires tuning

6. **Black box**: Why does this specific embedding make sense? Hard to explain

---

## Part 11: Variants and Extensions

### FastText (2017)

**Key innovation**: Represent words as **bag of character n-grams**

Example: "where"
- Character trigrams: <wh, whe, her, ere, re>
- Word embedding = sum of n-gram embeddings

**Advantages**:
- Handles rare words better
- Handles misspellings
- Generates embeddings for unseen words!
- Captures morphology (walk/walking/walked related)

### GloVe (2014)

**Key insight**: Combine global statistics (like LSA) with local context (like Word2Vec)

**Method**: 
- Build word co-occurrence matrix (global)
- Factorize it with a weighted least squares objective
- Goal: dot product of embeddings = log of co-occurrence probability

**Advantage**: Often better performance, especially on analogy tasks

### ELMo (2018)

**Revolution**: **Contextual** embeddings

Each word gets different embedding based on sentence:
- "I went to the **bank** to deposit money" → financial embedding
- "I sat by the river **bank**" → geographical embedding

**Method**: Deep bidirectional LSTM language model

### BERT (2018)

**Even bigger revolution**: Transformer-based contextual embeddings

- Uses attention mechanism (not recurrence)
- Bidirectional context
- Pre-trained on massive corpora
- Fine-tuned for specific tasks

**Word2Vec's legacy**: It pioneered the idea that embeddings should be learned from data, not hand-crafted!

---

## Part 12: Practical Usage

### Training Your Own Embeddings

**When to train**:
- Domain-specific corpus (medical, legal, technical)
- Non-English languages
- Lots of training data (millions of sentences)

**Libraries**:
- Gensim (Python): Easy, well-documented
- FastText (C++/Python): Fast, handles OOV
- TensorFlow/PyTorch: Full control

### Using Pre-trained Embeddings

**Popular sources**:
- Google's Word2Vec (3M words, 300d)
- GloVe (various sizes)
- FastText (157 languages)

**How to use**:
```python
from gensim.models import KeyedVectors

# Load pre-trained
model = KeyedVectors.load_word2vec_format('vectors.bin', binary=True)

# Get embedding
vec = model['cat']  # numpy array (300,)

# Find similar
similar = model.most_similar('cat', topn=10)
# Returns: [('dog', 0.87), ('kitten', 0.82), ...]

# Analogy
result = model.most_similar(positive=['woman', 'king'], 
                            negative=['man'], topn=1)
# Returns: [('queen', 0.71)]
```

### Fine-tuning Embeddings

Start with pre-trained, continue training on your corpus:
- Keeps general knowledge
- Adapts to your domain
- Requires less data than training from scratch

---

## Part 13: Evaluation

### Intrinsic Evaluation

**Analogy tasks**:
- Semantic: king:queen :: man:woman
- Syntactic: walk:walking :: swim:swimming

**Similarity tasks**:
- Correlation with human judgments
- WordSim-353, SimLex-999 datasets

### Extrinsic Evaluation

**Does it help downstream tasks?**
- Text classification
- Named entity recognition
- Sentiment analysis
- Machine translation

**This is what ultimately matters!**

---

## Summary: The Big Picture

Word2Vec revolutionized NLP by showing that:

1. **Neural networks can learn semantic representations** automatically from raw text
2. **Local context is enough** - don't need global statistics
3. **Simple objectives work** - predict context from word, or word from context
4. **Approximations are necessary** - negative sampling makes it practical
5. **Vector arithmetic captures relationships** - geometry reflects semantics

**The paradigm shift**: From hand-crafted features to learned representations.

**The foundation**: Word2Vec paved the way for modern NLP (BERT, GPT, etc.)

**The principle**: "A word is characterized by the company it keeps" - this distributional hypothesis, implemented through neural networks, unlocked machines' ability to understand language.